# COVID-19 in Ireland
> Dashboard

- toc: false
- branch: master
- badges: true
- comments: false
- image: images/covid-19-ie-cases-per-m.png
- permalink: /covid-19-dashboard-ie/
- author: "Libre AI"
- categories: [ireland, covid19]

In [1]:
#hide
import altair as alt
import pandas as pd
import numpy as np
import math

from IPython.display import Markdown, HTML

In [2]:
#hide
confirmed_all = pd.read_csv('data/time_series_covid19_confirmed_global.csv')
deaths_all = pd.read_csv('data/time_series_covid19_deaths_global.csv')
recovered_all = pd.read_csv('data/time_series_covid19_recovered_global.csv')

In [3]:
#hide

def dmy_to_isodate(dmy):
    m, d, y = dmy.split('/')
    return '{}-{:02d}-{:02d}'.format(2000 + int(y), int(m), int(d))
    
def get_count_data(df_all, country, start_count):
    df = df_all[df_all['Country/Region'] == country]
    counts = list(enumerate([(dmy_to_isodate(x[0]), x[1]) for x in list(zip(df.columns[4:], list(df.values[0][4:]))) if x[1] >= start_count]))
    days_since = [x for x,(_, _) in counts]
    dates = [x for _,(x, _)  in counts]
    freqs = [x for _,(_,x) in counts]
    return dates, freqs, days_since

def get_count_df(df_all, country, start_count, metric):
    dates, freqs, days_since = get_count_data(df_all, country, start_count)
    x = pd.DataFrame({
        'date': dates,
        'country': country,
        metric: freqs,
        'new_{}'.format(metric): np.hstack((np.array([0]), np.diff(freqs))),
        'days_since_{}_{}'.format(start_count, metric): days_since
    })
    return x
    
def get_confirmed_df(df_all, country, start_count=10):
    return get_count_df(df_all, country, start_count, 'cases')

def get_deaths_df(df_all, country, start_count=0):
    return get_count_df(df_all, country, start_count, 'deaths')

def get_recovered_df(df_all, country, start_count=0):
    return get_count_df(df_all, country, start_count, 'recovered')
    
def get_metrics_for_country(confirmed_all, deaths_all, recovered_all, country):
    cases = get_confirmed_df(confirmed_all, country)
    deaths = get_deaths_df(deaths_all, country)
    recovered = get_recovered_df(recovered_all, country)
    
    cd = pd.merge(cases, deaths, on=['date', 'country'], how='left')
    cdr = pd.merge(cd, recovered, on=['date', 'country'], how='left')
    return cdr
    
    
    

In [4]:
#hide
# confirmed, deaths, recovered (cdr) for each country
cdr_ie = get_metrics_for_country(confirmed_all, deaths_all, recovered_all, 'Ireland')
cdr_it = get_metrics_for_country(confirmed_all, deaths_all, recovered_all, 'Italy')
cdr_es = get_metrics_for_country(confirmed_all, deaths_all, recovered_all, 'Spain')

In [5]:
#hide
cdr_ie_latest = cdr_ie.tail(1)

cdr_ie_latest

,date,country,cases,new_cases,days_since_10_cases,deaths,new_deaths,days_since_0_deaths,recovered,new_recovered,days_since_0_recovered
26,2020-04-01,Ireland,3447,212,26,85,14,70,5,0,70


In [6]:
#hide

total_confirmed_ie = cdr_ie_latest['cases'].values[0]
total_deaths_ie = cdr_ie_latest['deaths'].values[0]
total_recovered_ie = cdr_ie_latest['recovered'].values[0]
(total_confirmed_ie, total_deaths_ie, total_recovered_ie)

(3447, 85, 5)

---

In [7]:
#hide_input
html_ie = '''
<h1>Total Confirmed: <span style="color:red">{}</span></h1>
<h1>Total Deaths: <span style="color:red">{}</span></h1>
<h1>Total Recovered: <span style="color:green">{}</span></h1>
'''.format(total_confirmed_ie, total_deaths_ie, total_recovered_ie)

HTML(html_ie)

---

In [8]:
#hide_input

from datetime import datetime
utc_now = datetime.utcnow()

Markdown('> Last updated on {} UTC'.format(utc_now.strftime("%Y-%m-%d %H:%M")))

> Last updated on 2020-04-02 05:36 UTC

In [9]:
#hide
cdr_it.tail(1)

,date,country,cases,new_cases,days_since_10_cases,deaths,new_deaths,days_since_0_deaths,recovered,new_recovered,days_since_0_recovered
40,2020-04-01,Italy,110574,4782,40,13155,727,70,16847,1118,70


In [10]:
#hide
cdr_es.tail(1)

,date,country,cases,new_cases,days_since_10_cases,deaths,new_deaths,days_since_0_deaths,recovered,new_recovered,days_since_0_recovered
35,2020-04-01,Spain,104118,8195,35,9387,923,70,22647,3388,70


In [11]:
#hide

#figures from https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population
population_in_m_ie = 4.9
population_in_m_it = 60.2
population_in_m_es = 47.1

In [12]:
#hide

def get_data_source(country, cdr, population_in_m):
    x = pd.DataFrame({
    'country': country,
    'date': cdr['date'],
    'days_since_10_cases': cdr['days_since_10_cases'],
    'confirmed_cases': cdr['cases'],
    'confirmed_cases_per_million': np.round(cdr['cases']/population_in_m),
    'new_cases': cdr['new_cases'],
    'new_deaths': cdr['new_deaths'],
    'new_recovered': cdr['new_recovered']
    })
    return x

source_ie = get_data_source('Ireland', cdr_ie, population_in_m_ie)

source_it = get_data_source('Italy', cdr_it, population_in_m_it)

source_es = get_data_source('Spain', cdr_es, population_in_m_es)


In [13]:
#hide

cases_ie_plot = alt.Chart(source_ie).mark_circle(
    color='green',
    opacity=0.7).encode(
    x='days_since_10_cases:Q',
#     y='confirmed_cases:Q',
    y=alt.Y('confirmed_cases:Q', scale=alt.Scale(type='sqrt')),
    size=alt.Size('new_cases:Q', title="Number of new cases in Ireland"),
    tooltip=['country', 'date:T', 'confirmed_cases', 'new_cases', 'new_deaths']
)#.interactive()

cases_ie_plot

alt.Chart(...)

In [14]:
#hide

cases_it_plot = alt.Chart(source_it).mark_line(
    color='blue',
    opacity=0.5,
    strokeDash=[4, 4],
    strokeWidth=3,
    point=True
).encode(
    x='days_since_10_cases:Q',
#     y='confirmed_cases:Q',
    y=alt.Y('confirmed_cases:Q', scale=alt.Scale(type='sqrt')),
#     size='new_cases:Q',
    tooltip=['country', 'date:T', 'confirmed_cases', 'new_cases', 'new_deaths'],
    color=alt.Color('country', scale=alt.Scale(range=['blue'], domain=['Italy']), title="Country")
)



cases_it_plot

alt.Chart(...)

In [15]:
#hide

cases_es_plot = alt.Chart(source_es).mark_line(
    color='red',
    opacity=0.5,
    strokeDash=[4,4],
    strokeWidth=3,
    point=True
).encode(
    x='days_since_10_cases:Q',
#     y='confirmed_cases:Q',
    y=alt.Y('confirmed_cases:Q', scale=alt.Scale(type='sqrt')),
#     size='new_cases:Q',
    tooltip=['country', 'date:T', 'confirmed_cases', 'new_cases', 'new_deaths'],
    color=alt.Color('country', scale=alt.Scale(range=['red'], domain=['Spain']), title="")
)

cases_es_plot

alt.Chart(...)

In [16]:
#hide
# # Number of persons tested positive for Covid-19 in Ireland
# The figure below shows the total number of persons who tested positive for Covid-19 (_cases_) in Ireland, Italy, and Spain. The `x` axis corresponds to the number of days since at least 10 cases were detected. This allow us to compare Ireland's current situation to the observed in Italy and Spain earlier. The comparison using Italy and Spain is because those countries are still in a critical situation fighting the pandemia, and no country would like to be the next one in such situation. 

# For Ireland, the size of the circles for each day denotes the number of new persons confirmed infected that day.

# **If we consider only the absolute numbers, we can see that the confirmed cases for Covid-19 in Ireland are below the ones observed in Italy and Spain at early stages. However, the populations of the three countries are very different and we have to account for that as well.**

In [17]:
#hide

alt.layer(cases_it_plot, cases_es_plot, cases_ie_plot).resolve_scale(
    color='independent',
    shape='independent'
)

alt.LayerChart(...)

---

In [18]:
#hide
# Number of Covid-19 cases per million

# Number of Covid-19 cases per million
The respective population of Ireland, Italy, and Spain is as follows (in millions) [\[source\]](https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population):

- **Ireland** : 4.9M
- **Italy** : 60.2M
- **Spain** : 47.1M

If we plot a figure dividing the total number of cases reported daily by the population (in millions) we can find how many persons are infected per 1 million, which give us a normalized value we could use to understand the severity of the outbreak.

### The figure below shows that in fact Ireland has more cases per million than the most affected countries in Europe starting on March 16th, 2020, and growing quickly.

In [19]:
#hide

cases_per_million_ie_plot = alt.Chart(source_ie).mark_circle(
    color='green',
    opacity=0.7).encode(
    x='days_since_10_cases:Q',
#     y='confirmed_cases:Q',
    y=alt.Y('confirmed_cases_per_million:Q', scale=alt.Scale(type='sqrt')),
    size=alt.Size('new_cases:Q', title="Number of new cases in Ireland"),
    tooltip=['country', 'date:T', 'confirmed_cases:Q', 'confirmed_cases_per_million:Q', 'new_cases', 'new_deaths']
)#.interactive()

cases_per_million_ie_plot

alt.Chart(...)

In [20]:
#hide

cases_per_million_it_plot = alt.Chart(source_it).mark_line(
    color='blue',
    opacity=0.5,
    strokeDash=[4, 4],
    strokeWidth=3,
    point=True
).encode(
    x='days_since_10_cases:Q',
#     y='confirmed_cases:Q',
    y=alt.Y('confirmed_cases_per_million:Q', scale=alt.Scale(type='sqrt')),
#     size='new_cases:Q',
    tooltip=['country', 'date:T', 'confirmed_cases:Q', 'confirmed_cases_per_million:Q', 'new_cases', 'new_deaths'],
    color=alt.Color('country', scale=alt.Scale(range=['blue'], domain=['Italy']), title="Country")
)

cases_per_million_it_plot

alt.Chart(...)

In [21]:
#hide

cases_per_million_es_plot = alt.Chart(source_es).mark_line(
    color='red',
    opacity=0.5,
    strokeDash=[4,4],
    strokeWidth=3,
    point=True
).encode(
    x='days_since_10_cases:Q',
#     y='confirmed_cases:Q',
    y=alt.Y('confirmed_cases_per_million:Q', scale=alt.Scale(type='sqrt')),
#     size='new_cases:Q',
    tooltip=['country', 'date:T', 'confirmed_cases:Q', 'confirmed_cases_per_million:Q', 'new_cases', 'new_deaths'],
    color=alt.Color('country', scale=alt.Scale(range=['red'], domain=['Spain']), title="")
)

cases_per_million_es_plot

alt.Chart(...)

In [22]:
#hide_input

alt.layer(cases_per_million_ie_plot, cases_per_million_it_plot, cases_per_million_es_plot).properties(
    width=800, height=600
).resolve_scale(
    color='independent',
    shape='independent'
)

alt.LayerChart(...)

---
### Data Sources
- Government of Ireland [https://www.gov.ie/]
- 2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE [https://github.com/CSSEGISandData/COVID-19]

---

<div align="right">Notebook and graphics by Dr. Ernesto Diaz-Aviles, <a href="https://twitter.com/vedax" target="_blank">@vedax</a><br/><a href="https://libreai.com" target="_blank">Libre AI</a><br/><a href="https://creativecommons.org/licenses/by-sa/4.0/" target="_blank">CC BY-SA 4.0</a>, 2020</div>

---